In [7]:
import tensorflow as tf
import os
import cv2
import pickle
from tqdm import tqdm
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [8]:
def extract_images_from_folder(root_folder, target_size=(256, 256)):
    image_data = []
    labels = []
    label_to_index = {}  # Dictionary to map label names to numerical indices
    index = 0

    for subdir, dirs, files in os.walk(root_folder):
        for file in tqdm(files):
            filepath = os.path.join(subdir, file)
            img = cv2.imread(filepath)
            img = cv2.resize(img, target_size)

            label = os.path.basename(subdir)
            if label not in label_to_index:
                label_to_index[label] = index
                index += 1

            image_data.append(img)
            labels.append(label_to_index[label])

    image_data = np.array(image_data)
    labels = np.array(labels)

    return image_data, labels, label_to_index

In [9]:
root_folder = "D:\miniprojects\dl\Train\Train"
x_train,y_train,label_reverse_train = extract_images_from_folder(root_folder)

0it [00:00, ?it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 434/434 [00:35<00:00, 12.34it/s]


In [10]:
root_folder = "D:\miniprojects\dl\Validation\Validation"
x_val,y_val,label_reverse_val = extract_images_from_folder(root_folder)

0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 11.10it/s]


In [11]:
root_folder = "D:\miniprojects\dl\Test\Test"
x_test,y_test,label_reverse_test = extract_images_from_folder(root_folder)

0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 13.64it/s]


In [12]:
x_train = x_train / 255.0
x_test = x_test / 255.0
x_val = x_val / 255.0

In [13]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_reverse_train), activation='softmax')
])

D:\anaconda\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [14]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 36s 783ms/step - accuracy: 0.4537 - loss: 1.2126 - val_accuracy: 0.5167 - val_loss: 0.9275
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 761ms/step - accuracy: 0.6792 - loss: 0.6755 - val_accuracy: 0.6667 - val_loss: 0.6217
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 758ms/step - accuracy: 0.7662 - loss: 0.5318 - val_accuracy: 0.7333 - val_loss: 0.5959
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 33s 774ms/step - accuracy: 0.8637 - loss: 0.3969 - val_accuracy: 0.8000 - val_loss: 0.5106
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 754ms/step - accuracy: 0.9289 - loss: 0.2456 - val_accuracy: 0.8000 - val_loss: 0.4140
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 756ms/step - accuracy: 0.9243 - loss: 0.2572 - val_accuracy: 0.8833 - val_loss: 0.3221
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 754ms/step - accuracy: 0.9484 - loss: 0.1639 - val_accuracy: 0.9000 - val_loss: 0.3322
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 32s 758ms/step - accuracy: 0.9355 - loss: 0.2086 - val_accu

In [16]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 208ms/step - accuracy: 0.8965 - loss: 0.5241
Test accuracy: 0.8600000143051147


In [ ]:
#https://www.kaggle.com/datasets/rashikrahmanpritom/plant-disease-recognition-dataset